In [1]:
# Importing libraries
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
# creating a dataframe
data = {'returns':[.1, -.1, .1], 'vol':[.1, .2, .3],'weights':[.33, .33, .33]} 
df = pd.DataFrame(data, index =['x', 'y', 'z']) 

In [5]:
# Lets assume that we have invested 100 dollars in the portfolio
df['dollar_returns'] = df['returns']*100
df['standard_dev']= df['vol']
df

,returns,vol,weights,dollar_returns,standard_dev
x,0.1,0.1,0.33,10.0,0.1
y,-0.1,0.2,0.33,-10.0,0.2
z,0.1,0.3,0.33,10.0,0.3


In [6]:
# Assumptions and calculations
risk_free_rate = 0.0178
weights = np.array(df['weights'])
returns = np.array(df['returns'])
volatility = np.array(df['vol'])
sd = np.array(df['standard_dev'])
portfolio_returns = np.dot(weights,returns)
portfolio_vol = volatility.mean()
portfolio_standard_dev = sd.mean()
cov = np.cov(returns)
num_portfolios = 2500

In [7]:
sharpe_ratio = (portfolio_returns - risk_free_rate)/ portfolio_standard_dev
sharpe_ratio

0.076

In [8]:
import scipy.optimize as sco
from scipy import stats

### Calculating the max sharpe ratio for portfolio optimization

### A) Fully invested long-only portfolio

In [9]:
def calc_neg_sharpe(weights, returns, cov, risk_free_rate):
    portfolio_return = np.sum(returns * weights)
    portfolio_std = portfolio_standard_dev
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    return -sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

def max_sharpe_ratio(returns, cov, risk_free_rate):
    num_assets = len(returns)
    args = (returns, cov, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(calc_neg_sharpe, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

optimal_port_sharpe = max_sharpe_ratio(returns, cov, risk_free_rate)

In [10]:
pd.DataFrame([round(x,2) for x in optimal_port_sharpe['x']]).T

,0,1,2
0,0.5,0.0,0.5


### B) Long/short portfolio with max 200% gross

In [11]:
def calc_neg_sharpe(weights, returns, cov, risk_free_rate):
    portfolio_return = np.sum(returns * weights)
    portfolio_std = portfolio_standard_dev
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    return -sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x*2) - 1})

def max_sharpe_ratio(returns, cov, risk_free_rate):
    num_assets = len(returns)
    args = (returns, cov, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x*2) - 1})
    bound = (-1.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(calc_neg_sharpe, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

optimal_port_sharpe2 = max_sharpe_ratio(returns, cov, risk_free_rate)

In [12]:
pd.DataFrame([round(x,2) for x in optimal_port_sharpe2['x']]).T

,0,1,2
0,0.75,-1.0,0.75
